In [67]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [68]:
df = pd.read_csv("../cleaning-data/api-cleaned-data.csv")
df.head()

,flight no,date,booster version,payload mass,orbit,launch site,landing type,outcome,flights,gridfins,...,legs,landing pad,block,reused count,serial,longitude,latitude,cost per launch,customers,manufacterers
0,6,2010-06-04,Falcon 9,8117.574038,LEO,Cape Canaveral Space Force Station Space Launc...,NaN,False,1,False,...,False,NaN,1.0,0,B0003,-80.577366,28.561857,50000000,SpaceX,SpaceX
1,7,2010-12-08,Falcon 9,8117.574038,LEO,Cape Canaveral Space Force Station Space Launc...,NaN,False,1,False,...,False,NaN,1.0,0,B0004,-80.577366,28.561857,50000000,NASA(COTS),SpaceX
2,8,2012-05-22,Falcon 9,525.000000,LEO,Cape Canaveral Space Force Station Space Launc...,NaN,False,1,False,...,False,NaN,1.0,0,B0005,-80.577366,28.561857,50000000,NASA(COTS),SpaceX
3,9,2012-10-08,Falcon 9,400.000000,ISS,Cape Canaveral Space Force Station Space Launc...,NaN,False,1,False,...,False,NaN,1.0,0,B0006,-80.577366,28.561857,50000000,NASA (CRS),SpaceX
4,10,2013-03-01,Falcon 9,677.000000,ISS,Cape Canaveral Space Force Station Space Launc...,NaN,False,1,False,...,False,NaN,1.0,0,B0007,-80.577366,28.561857,50000000,NASA (CRS),SpaceX


1. Mark all launch sites on a map

We have 3 main launch sites:

* CCSFS SLC-40: this site is used much for R&D (Cape Canaveral Space Force Station Space Launch Complex 40)

* VSFB SLC-4E (Vandenberg Space Force Base Space Launch Complex 4E)

* KSC LC-39A (Kennedy Space Center Historic Launch Complex 39A)

Some collected insights:

* When we zoom out the map, we can see that VAFB launch site is located on the west coast, CCSFS and KSC are near together which are located on the east coast, NASA Johnson Space Center are in the middle of 2 coast, located nea Houston, Texas

* All launch sites are in proximity to equator. This makes sense as it takes less fuel to get into space from the equator due to the physics of Earth's rotation. 

* The launch sites in close proximity to the coast are also logical for safety reasons.

In [69]:
df['launch site'].replace("Cape Canaveral Space Force Station Space Launch Complex 40", "CCSFS SLC-40", inplace=True)
df['launch site'].replace("Vandenberg Space Force Base Space Launch Complex 4E", "VSFB SLC-4E", inplace=True)
df['launch site'].replace("Kennedy Space Center Historic Launch Complex 39A", "KSC LC-39A", inplace=True)
spacex_df = df[['launch site', 'longitude', 'latitude', 'outcome']]

In [70]:
spacex_df.head()

,launch site,longitude,latitude,outcome
0,CCSFS SLC-40,-80.577366,28.561857,False
1,CCSFS SLC-40,-80.577366,28.561857,False
2,CCSFS SLC-40,-80.577366,28.561857,False
3,CCSFS SLC-40,-80.577366,28.561857,False
4,CCSFS SLC-40,-80.577366,28.561857,False


In [71]:
launch_site_info = spacex_df[['launch site', 'longitude', 'latitude']]
launch_site_info = launch_site_info.groupby('launch site', as_index=False).first()
launch_site_info

,launch site,longitude,latitude
0,CCSFS SLC-40,-80.577366,28.561857
1,KSC LC-39A,-80.603956,28.608058
2,VSFB SLC-4E,-120.610829,34.632093


In [72]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(nasa_coordinate, zoom_start=5)
circle = folium.Circle(nasa_coordinate, radius=1000, color="#404E7C", fill=True)
circle = circle.add_child(folium.Popup('NASA Johnson Space Center'))
marker = folium.map.Marker(
    nasa_coordinate,
    icon = DivIcon(icon_size=(20,20), 
                   icon_anchor=(0,0), 
                   html='<div style="font-size: 12; color: #404E7C;"><b>%s</b></div>' % 'NASA JSC')
)
site_map.add_child(circle)
site_map.add_child(marker)

In [73]:
for index, row in launch_site_info.iterrows():
    coordinate = [row[2], row[1]]
    launchsite = row[0]
    circle = folium.Circle(coordinate, radius=1000, color='#69140E', fill=True)
    circle = circle.add_child(folium.Popup(launchsite))
    marker = folium.map.Marker(
        coordinate,
        icon = DivIcon(icon_size=(20,20),
                       icon_anchor=(0,0),
                       html='<div style="color: #69140E; font-size: 12"><b>%s</b></div>' % launchsite)
    )
    site_map.add_child(circle)
    site_map.add_child(marker)

In [74]:
site_map

2. Mark the success/failed launches for each site on the map

Mark the success landing as yellow color and failure landing as red color

* Success status: <div style="width: 20px; height: 20px; background: #ECC30B; border-radius: 50%"></div>

* Failure status: <div style="width: 20px; height: 20px; background: #A8201A; border-radius: 50%"></div>

In [75]:
spacex_df.head()

,launch site,longitude,latitude,outcome
0,CCSFS SLC-40,-80.577366,28.561857,False
1,CCSFS SLC-40,-80.577366,28.561857,False
2,CCSFS SLC-40,-80.577366,28.561857,False
3,CCSFS SLC-40,-80.577366,28.561857,False
4,CCSFS SLC-40,-80.577366,28.561857,False


In [76]:
marker_cluster = MarkerCluster()
site_map.add_child(marker_cluster)
for index, row in spacex_df.iterrows():
    coordinate = [row[2], row[1]]
    color = '#ECC30B' if row[3] == True else '#A8201A'
    folium.map.Marker(
        coordinate, 
        icon = folium.Icon(color='white',
                           icon_color=color)
    ).add_to(marker_cluster)

In [77]:
site_map

3. Calculate the distances between a launch site to its proximities

* Use MousePosition on the map to get coordinate for a mouse over a point on the map: choose the railway coordinate near the VSFB launch site: latitude: 34.68575, longitude: -120.58971.

* The distance from VSFB to its railway is 6.27km

* Launch site are in close proximity to coastline, which can help rocket fly over the ocean during launch, crew also has option to abort launch and attempt water landing, and finally minimize the risk from falling debris

* Also launch site need to near the railway and highway which allows easily transport for heavy cargo and property

* Launch site is far from city to restrict the danger of launching rockets to population dense areas

In [78]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    prefix='Lat: ',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    lat_formatter=formatter,
    lng_formatter=formatter
)
site_map.add_child(mouse_position)

In [79]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lng1, lat2, lng2):
    R = 6373.0 # approximate radius of earth in km
    lat1 = radians(lat1)
    lng1 = radians(lng1)
    lat2 = radians(lat2)
    lng2 = radians(lng2)
    distance_lng = lng2 - lng1
    distance_lat = lat2 - lat1
    a = sin(distance_lat/2)**2 + cos(lat1) * cos(lat2) * sin(distance_lng/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

In [88]:
VSFB_coordinator = [34.632093, -120.610829]
railway_coordinator = [34.68575, -120.58971]
result = calculate_distance(VSFB_coordinator[0], VSFB_coordinator[1], railway_coordinator[0], railway_coordinator[1])
result

6.273245984474221

In [90]:
distance_marker = folium.Marker(
    railway_coordinator,
    icon = DivIcon(icon_size=(20,20),
                   icon_anchor=(0,0),
                   html='<div style="font-size: 14; color: #A8201A;"><b>%s</b></div>' % "{:10.2f} KM".format(result))
)
site_map.add_child(distance_marker)

In [91]:
lines = folium.PolyLine(locations=[VSFB_coordinator, railway_coordinator], weight=2)
site_map.add_child(lines)